In [5]:
import os
os.environ["OMP_NUM_THREADS"] = "32"

In [13]:
from graph_tool.all import *
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.covariance import LedoitWolf, OAS
import matplotlib.pyplot as plt
import matplotlib.cm as mpl


import statsmodels.api as sm
from multipy.fdr import qvalue
from multipy.fdr import lsu

import dill

import fit_sbm
import trim_networks
import export_blocks

In [14]:
 g = load_graph("../data/output/SBM/gtex/graph/2_SKIN.xml.gz")

In [16]:
g

<Graph object, undirected, with 5918 vertices and 17508403 edges, 1 internal vertex property, 2 internal edge properties, at 0x7f9cbdd609d0>

In [34]:
gi = trim_networks.filterByFDR(g, 0.01, True)
N = len(gi.get_vertices())
print(str(N) + " genes")
Et = (N * N - N)/2
E = len(gi.get_edges())
density = E/Et        
print("Density: " + str(density))


5918 genes
Density: 0.7514606557776857


In [37]:
corr = gi.edge_properties["spearman"]
corr
gi.ep.z_s = gi.new_edge_property("double", (2*np.arctanh(corr.a)))

In [40]:
state_min = minimize_nested_blockmodel_dl(gi, init_bs=None,
                                          state_args=dict(recs=[gi.ep.z_s],
                                          rec_types=["real-normal"]))

In [42]:
S1 = state_min.entropy()
print("Starting annealing...")
mcmc_anneal(state_min, beta_range=(1, 10), niter=100,
            mcmc_equilibrate_args=dict(force_niter=100))
S2 = state_min.entropy()
print("Improvement from annealing:", S2 - S1)
print("Final entropy after annealing: " + str(state_min.entropy()))

Starting annealing...
Improvement from annealing: -2846369.330363484
Final entropy after annealing: 14586723.511638353


In [ ]:
state_min.draw()